# Install all the `dependancies` which is required to run program 

1. labelme          -`to create JSON file containing all info of image label`    
2. tensorflow       - `required library`
3. tensorflow-gpu   - `required library`
4. pencv-python     - ` to visualize and draw rectangular`
5. matplotlib       - `to visualize loaded images` 
6. albumentations   - `to do data augmetation`

In [ ]:
# `Import required libraries`
import os
import time
import uuid
import cv2
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
import PIL

# Save at least 20 Images of face in  Image path  

In [ ]:
IMAGES_PATH = '.\data\images'

# use label me to create JSON file for Image label

In [ ]:
!labelme

# load images in tensorflow data pipeline

In [ ]:
images = tf.data.Dataset.list_files('data\\images\\*.jpg')

def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img


images = images.map(load_image)
images.as_numpy_iterator().next()

Create pathes: 

data\train\images -`for train images`

data\train\labels - `for train labels`

as well as for test and validation data set.



now transfer all image file and labels file to these folders from main folder where you have saved 20 images and their labels using labelme

# create more data with augmentation

In [ ]:
import albumentations as alb
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450),
                         alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2),
                         alb.RGBShift(p=0.2),
                         alb.VerticalFlip(p=0.5)],
                        bbox_params=alb.BboxParams(format='albumentations',
                                                   label_fields=['class_labels']))


# appply `augmentor to all images in train test and val`

In [ ]:
for set in ['train', 'test', 'val']:
    for image in os.listdir(os.path.join('data', set, 'images')):
        img = cv2.imread(os.path.join('data', set, 'images', image))

        coords = [0, 0, 0.00001, 0.00001]
        label_path = os.path.join('data', set, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640, 480, 640, 480]))

        try:
            for x in range(60):
                augmented = augmentor(
                    image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('aug_data', set, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0:
                        annotation['bbox'] = [0, 0, 0, 0]
                        annotation['class'] = 0
                    else:
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else:
                    annotation['bbox'] = [0, 0, 0, 0]
                    annotation['class'] = 0

                with open(os.path.join('aug_data', set, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)


In [ ]:

aug_data = dict()
for set in ['train', 'test', 'val']:
    aug_data[set] = tf.data.Dataset.list_files(os.path.join('aug_data',set,'images','*.jpg'), shuffle=False)
    aug_data[set] = aug_data[set].map(load_image)
    aug_data[set] = aug_data[set].map(lambda x: tf.image.resize(x, (120, 120)))
    aug_data[set] = aug_data[set].map(lambda x: x/255)


In [ ]:
aug_data['train'].as_numpy_iterator().next()

# now process to prepare the label 

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding="utf-8") as f:
        label = json.load(f)

    return [label['class']], label['bbox']


In [ ]:

for i in ['train','test','val']:
    aug_data[i +'labels'] = tf.data.Dataset.list_files(os.path.join('aug_data',i,'labels','*.json'), shuffle=False)
    aug_data[i+ 'labels'] = aug_data[i+ 'labels'].map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))


In [ ]:
aug_data['trainlabels'].as_numpy_iterator().next()


In [ ]:
len(aug_data['train']), len(aug_data['trainlabels']), len(aug_data['test']), len(aug_data['testlabels']), len(aug_data['val']), len(aug_data['vallabels'])


# create dataset of 8 images per batch with labels 

In [ ]:
train = tf.data.Dataset.zip((aug_data['train'], aug_data['trainlabels']))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

test = tf.data.Dataset.zip((aug_data['test'], aug_data['testlabels']))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

val = tf.data.Dataset.zip((aug_data['val'], aug_data['vallabels']))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)


In [ ]:
train.as_numpy_iterator().next()[1]


# view images 

In [ ]:
data_samples = train.as_numpy_iterator()


In [ ]:
res = data_samples.next()
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
for idx in range(4):
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]

    cv2.rectangle(sample_image,
                  tuple(np.multiply(sample_coords[:2], [
                        120, 120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [
                        120, 120]).astype(int)),
                  (255, 0, 0), 2)

    ax[idx].imshow(sample_image)


# build model with VGG api


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
vgg = VGG16(include_top=False)
vgg.summary()


In [ ]:
def build_model():
    input_layer = Input(shape=(120, 120, 3))

    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(1, activation='sigmoid')(class1)

    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)

    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker


In [ ]:
facetracker = build_model()
facetracker.summary()


In [ ]:
X, y = train.as_numpy_iterator().next()
classes, coords = facetracker.predict(X)
classes, coords


# loss and optimiser 

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)


In [ ]:
def localization_loss(y_true, yhat):
    delta_coord = tf.reduce_sum(tf.square(y_true[:, :2] - yhat[:, :2]))

    h_true = y_true[:, 3] - y_true[:, 1]
    w_true = y_true[:, 2] - y_true[:, 0]

    h_pred = yhat[:, 3] - yhat[:, 1]
    w_pred = yhat[:, 2] - yhat[:, 0]

    delta_size = tf.reduce_sum(
        tf.square(w_true - w_pred) + tf.square(h_true-h_pred))

    return delta_coord + delta_size


In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss


In [ ]:
localization_loss(y[1], coords)


In [ ]:
regressloss(y[1], coords)


# time to start training model 

In [ ]:
class FaceTracker(Model):
    def __init__(self, eyetracker,  **kwargs):
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def train_step(self, batch, **kwargs):

        X, y = batch

        with tf.GradientTape() as tape:
            classes, coords = self.model(X, training=True)

            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(
                tf.cast(y[1], tf.float32), coords)

            total_loss = batch_localizationloss+0.5*batch_classloss

            grad = tape.gradient(total_loss, self.model.trainable_variables)

        opt.apply_gradients(zip(grad, self.model.trainable_variables))

        return {"total_loss": total_loss, "class_loss": batch_classloss, "regress_loss": batch_localizationloss}

    def test_step(self, batch, **kwargs):
        X, y = batch

        classes, coords = self.model(X, training=False)

        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss

        return {"total_loss": total_loss, "class_loss": batch_classloss, "regress_loss": batch_localizationloss}

    def call(self, X, **kwargs):
        return self.model(X, **kwargs)


In [ ]:
model = FaceTracker(facetracker)
model.compile(opt, classloss, regressloss)


In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=10, validation_data=val,
                 callbacks=[tensorboard_callback])


In [ ]:
hist.history


In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20, 5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'],
           color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'],
           color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()


# save the model 

In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
facetracker.save('facetracker.h5')
facetracker = load_model('facetracker.h5')


# useyour model and detect real time 

In [ ]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    _, frame = cap.read()
    frame = frame[50:500, 50:500, :]

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120, 120))

    yhat = facetracker.predict(np.expand_dims(resized/255, 0))
    sample_coords = yhat[1][0]

    if yhat[0] > 0.5:
        # Controls the main rectangle
        cv2.rectangle(frame,
                      tuple(np.multiply(sample_coords[:2], [
                            450, 450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [
                            450, 450]).astype(int)),
                      (255, 0, 0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame,
                      tuple(np.add(np.multiply(sample_coords[:2], [450, 450]).astype(int),
                                   [0, -30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450, 450]).astype(int),
                                   [80, 0])),
                      (255, 0, 0), -1)

        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450, 450]).astype(int),
                                                [0, -5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('EyeTrack', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
